In [ ]:
# Notebook 6: Aggregated Answer and Usage Example

# Run the previous notebooks to load environment variables, imports, embeddings, chunked texts, vector stores, RAG chains, and conversational RAG chains
%run "environment_setup.ipynb"
%run "embedding_and_chunking.ipynb"
%run "vector_store_creation.ipynb"
%run "retriever_and_ragchain_creation.ipynb"
%run "conversational_ragchain.ipynb"
def get_aggregated_answer_and_sources(input_question, session_id="abc123"):
    def is_invalid_answer(answer):
        return "I don't" in answer or "I'm sorry" in answer

    def get_response_and_sources(response, all_texts):
        answer = response["answer"]
        context = response["context"]
        used_chunks = [doc for doc in all_texts if any(chunk.page_content in doc.page_content for chunk in context)]
        sources = {(chunk.metadata['file'], chunk.metadata.get('page', 'Unknown')) for chunk in used_chunks}
        return answer, context, sources

    all_responses = {}
    all_contexts = []
    all_sources = set()

    for key in ['chroma', 'faiss', 'pinecone']:
        response = conversational_rag_chains[key].invoke(
            {"input": input_question},
            config={"configurable": {"session_id": session_id}},
        )
        if not is_invalid_answer(response["answer"]):
            answer, context, sources = get_response_and_sources(response, all_texts[key])
            all_responses[key] = answer
            all_contexts.extend(context)
            all_sources.update(sources)

    if not all_responses:
        return "I don't know the answer.", {}, [], set()

    # Combine all the valid responses into one, avoiding duplicates
    unique_answer_parts = set()
    for answer in all_responses.values():
        unique_answer_parts.update(answer.split(". "))

    aggregated_answer = ". ".join(sorted(unique_answer_parts))

    return aggregated_answer, all_responses, all_contexts, all_sources

def format_context(context: List[Any]) -> str:
    formatted_context = "\n\n".join([
        f"File: {doc.metadata['file']}, Page {doc.metadata.get('page', 'Unknown')}: {doc.page_content}"
        for doc in context
    ])
    return formatted_context

# Usage Example
question = "What are the primary tasks performed by broadcast technicians during radio and television broadcasts?"
aggregated_answer, all_responses, context, sources = get_aggregated_answer_and_sources(question)

# Format the output in a readable way
formatted_context = "\n\n".join([f"File: {doc.metadata['file']}, Page {doc.metadata.get('page', 'Unknown')}: {doc.page_content}" for doc in context])

output = f"""
Final Aggregated Answer: {aggregated_answer}

Intermediate Answers:
Chroma: {all_responses.get('chroma', 'No valid answer found')}
FAISS: {all_responses.get('faiss', 'No valid answer found')}
Pinecone: {all_responses.get('pinecone', 'No valid answer found')}

Context used for answer:
{formatted_context}
"""

print(output)



C:\Users\vlad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\vlad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]